In [2]:
import pandas as pd

PATH = './materials/00_materials/COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'

doc = pd.read_csv(PATH  + '01-22-2020.csv', encoding='utf-8-sig')
try:
    doc = doc[['Province_State', 'Country_Region', 'Confirmed']] # 특정 컬럼만 선택하여 데이터 프레임 만들기
except: 
    doc = doc[['Province/State', 'Country/Region', 'Confirmed']] # 데이터 컬럼이 통일 되지 않은 경우 에러처리
    doc.columns = ['Province/State', 'Country_Region', 'Confirmed'] # 컬럼명 지정

doc = doc.dropna(subset=['Confirmed']) # 없는 데이터 삭제
doc = doc.astype({'Confirmed': 'int64'}) # 데이터 형변환
doc.head()

,Province/State,Country_Region,Confirmed
0,Anhui,Mainland China,1
1,Beijing,Mainland China,14
2,Chongqing,Mainland China,6
3,Fujian,Mainland China,1
5,Guangdong,Mainland China,26


In [3]:
# 중복 데이터 합치기
doc.groupby('Country_Region').sum()   # Province/State의 경우 문자열이기 때문에 삭제

,Confirmed
Country_Region,
Japan,2
Macau,1
Mainland China,547
South Korea,1
Taiwan,1
Thailand,2
US,1


In [9]:
# 지금까지 진행 한 내용을 함수로 정리:
import pandas as pd
import json

PATH = './materials/00_materials/COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'

with open('./materials/00_materials/COVID-19-master/csse_covid_19_data/country_convert.json', 'r', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)
    
def country_name_convert(row):
    if row['Country_Region'] in json_data:
        return json_data[row['Country_Region']]
    return row['Country_Region']

def create_dateframe(filename):
    
    doc = pd.read_csv(PATH + filename, encoding='utf-8-sig') 
    try:
        doc = doc[['Country_Region', 'Confirmed']]
    except:
        doc = doc[['Country/Region', 'Confirmed']]
        doc.columns = ['Country_Region', 'Confirmed']
    doc = doc.dropna(subset=['Confirmed'])
    doc['Country_Regeion'] = doc.apply(country_name_convert, axis=1) # Countyr_Region의 국가명을 여러 파일에 일관된 형식으로 변경
    doc = doc.astype({'Confirmed': 'int64'})
    doc = doc.groupby('Country_Region').sum()
    
    # 파일명을 기반으로 날짜 문자열 변환, Confirmed 컬럼명 변경
    date_column = filename.split('.')[0].lstrip('0').replace('-', '/')
    doc.columns = [date_column]
    return doc

In [10]:
# 테스트
doc1 = create_dateframe('01-22-2020.csv')
doc2 = create_dateframe('04-01-2020.csv')

In [11]:
doc2.head()

,4/01/2020
Country_Region,
Afghanistan,237
Albania,259
Algeria,847
Andorra,390
Angola,8


In [13]:
doc = pd.merge(doc1, doc2, how='outer', left_index=True, right_index=True)
doc.head()

,1/22/2020,4/01/2020
Country_Region,,
Afghanistan,NaN,237.0
Albania,NaN,259.0
Algeria,NaN,847.0
Andorra,NaN,390.0
Angola,NaN,8.0


In [14]:
# 결측치 제거
doc = doc.fillna(0)
doc

,1/22/2020,4/01/2020
Country_Region,,
Afghanistan,0.0,237.0
Albania,0.0,259.0
Algeria,0.0,847.0
Andorra,0.0,390.0
Angola,0.0,8.0
...,...,...
Venezuela,0.0,143.0
Vietnam,0.0,218.0
West Bank and Gaza,0.0,134.0


In [15]:
# 폴더에 있는 파일 목록 가져오기
import os
#file_list = os.listdir(PATH)
#csv_list = list()
# -->  한 줄로도 가능하다
file_list, csv_list = os.listdir(PATH), list()

for file in file_list:
    if file.split('.')[-1] == 'csv':
        csv_list.append(file)
        
print(csv_list)


['01-22-2020.csv', '01-23-2020.csv', '01-24-2020.csv', '01-25-2020.csv', '01-26-2020.csv', '01-27-2020.csv', '01-28-2020.csv', '01-29-2020.csv', '01-30-2020.csv', '01-31-2020.csv', '02-01-2020.csv', '02-02-2020.csv', '02-03-2020.csv', '02-04-2020.csv', '02-05-2020.csv', '02-06-2020.csv', '02-07-2020.csv', '02-08-2020.csv', '02-09-2020.csv', '02-10-2020.csv', '02-11-2020.csv', '02-12-2020.csv', '02-13-2020.csv', '02-14-2020.csv', '02-15-2020.csv', '02-16-2020.csv', '02-17-2020.csv', '02-18-2020.csv', '02-19-2020.csv', '02-20-2020.csv', '02-21-2020.csv', '02-22-2020.csv', '02-23-2020.csv', '02-24-2020.csv', '02-25-2020.csv', '02-26-2020.csv', '02-27-2020.csv', '02-28-2020.csv', '02-29-2020.csv', '03-01-2020.csv', '03-02-2020.csv', '03-03-2020.csv', '03-04-2020.csv', '03-05-2020.csv', '03-06-2020.csv', '03-07-2020.csv', '03-08-2020.csv', '03-09-2020.csv', '03-10-2020.csv', '03-11-2020.csv', '03-12-2020.csv', '03-13-2020.csv', '03-14-2020.csv', '03-15-2020.csv', '03-16-2020.csv', '03-17-20

In [16]:
# 정렬 .sort() 오름차순(default)        .sort(reverse=True) 내림차순

csv_list.sort()
csv_list

['01-22-2020.csv',
 '01-23-2020.csv',
 '01-24-2020.csv',
 '01-25-2020.csv',
 '01-26-2020.csv',
 '01-27-2020.csv',
 '01-28-2020.csv',
 '01-29-2020.csv',
 '01-30-2020.csv',
 '01-31-2020.csv',
 '02-01-2020.csv',
 '02-02-2020.csv',
 '02-03-2020.csv',
 '02-04-2020.csv',
 '02-05-2020.csv',
 '02-06-2020.csv',
 '02-07-2020.csv',
 '02-08-2020.csv',
 '02-09-2020.csv',
 '02-10-2020.csv',
 '02-11-2020.csv',
 '02-12-2020.csv',
 '02-13-2020.csv',
 '02-14-2020.csv',
 '02-15-2020.csv',
 '02-16-2020.csv',
 '02-17-2020.csv',
 '02-18-2020.csv',
 '02-19-2020.csv',
 '02-20-2020.csv',
 '02-21-2020.csv',
 '02-22-2020.csv',
 '02-23-2020.csv',
 '02-24-2020.csv',
 '02-25-2020.csv',
 '02-26-2020.csv',
 '02-27-2020.csv',
 '02-28-2020.csv',
 '02-29-2020.csv',
 '03-01-2020.csv',
 '03-02-2020.csv',
 '03-03-2020.csv',
 '03-04-2020.csv',
 '03-05-2020.csv',
 '03-06-2020.csv',
 '03-07-2020.csv',
 '03-08-2020.csv',
 '03-09-2020.csv',
 '03-10-2020.csv',
 '03-11-2020.csv',
 '03-12-2020.csv',
 '03-13-2020.csv',
 '03-14-2020

In [17]:
# 함수로 변경
import os 

def generate_dateframe_by_path(PATH):
    
    file_list, csv_list = os.listdir(PATH), list()
    first_doc = True
    for file in file_list:
        if file.split('.')[-1] == 'csv':
            csv_list.append(file)
    csv_list.sort()
    
    for file in csv_list:
        doc = create_dateframe(file)
        if first_doc:                          # 1번만 실행
            final_doc, first_doc = doc, False  
        else:
            final_doc = pd.merge(final_doc, doc, how='outer', left_index=True, right_index=True)
            
            
    final_doc = final_doc.fillna(0)
    return final_doc

In [19]:
doc = generate_dateframe_by_path(PATH)
doc

,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,...,6/08/2020,6/09/2020,6/10/2020,6/11/2020,6/12/2020,6/13/2020,6/14/2020,6/15/2020,6/16/2020,6/17/2020
Country_Region,,,,,,,,,,,,,,,,,,,,,
Azerbaijan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,20917.0,21459.0,22142.0,22890.0,23546.0,24102.0,24766.0,25527.0,26310.0,26874.0
Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1263.0,1299.0,1341.0,1385.0,1416.0,1464.0,1521.0,1590.0,1672.0,1722.0
Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10265.0,10382.0,10484.0,10589.0,10698.0,10810.0,10919.0,11031.0,11147.0,11268.0
Andorra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,852.0,852.0,852.0,852.0,853.0,853.0,853.0,853.0,854.0,854.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Western Sahara,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0
Yemen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,496.0,524.0,560.0,591.0,632.0,705.0,728.0,844.0,885.0,902.0
Zambia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1200.0,1200.0,1200.0,1200.0,1321.0,1357.0,1358.0,1382.0,1405.0,1412.0


In [20]:
doc = doc.astype('int64')
doc

,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,...,6/08/2020,6/09/2020,6/10/2020,6/11/2020,6/12/2020,6/13/2020,6/14/2020,6/15/2020,6/16/2020,6/17/2020
Country_Region,,,,,,,,,,,,,,,,,,,,,
Azerbaijan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,20917,21459,22142,22890,23546,24102,24766,25527,26310,26874
Albania,0,0,0,0,0,0,0,0,0,0,...,1263,1299,1341,1385,1416,1464,1521,1590,1672,1722
Algeria,0,0,0,0,0,0,0,0,0,0,...,10265,10382,10484,10589,10698,10810,10919,11031,11147,11268
Andorra,0,0,0,0,0,0,0,0,0,0,...,852,852,852,852,853,853,853,853,854,854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Western Sahara,0,0,0,0,0,0,0,0,0,0,...,9,9,9,9,9,9,9,9,9,9
Yemen,0,0,0,0,0,0,0,0,0,0,...,496,524,560,591,632,705,728,844,885,902
Zambia,0,0,0,0,0,0,0,0,0,0,...,1200,1200,1200,1200,1321,1357,1358,1382,1405,1412


In [21]:
# 변환한 데이터 파일로 쓰기
doc.to_csv('./final_df.csv')